In [ ]:
#	Prompt LLMs to be expert
#       they can now respond to question and provide structured response
#	Generate domain-specific multiple choice questions
#       on the way
#	Grade the result of LLM and find out correlations
#       with strcuture output, should be easy


In [1]:
import json
import io
import pandas as pd
from phi.agent import Agent, RunResponse
from phi.model.ollama import Ollama
from phi.storage.agent.sqlite import SqlAgentStorage
import requests

In [2]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [3]:
# global variables
model_id = "llama3.1"
#qwen2.5-coder, llama3.1, llama3.2


In [4]:
# gload occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [5]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(5).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
310,251125,"History Teachers, Postsecondary",Teach courses in human history and historiogra...
351,259044,"Teaching Assistants, Postsecondary",Assist faculty or other instructional staff in...
712,435031,Public Safety Telecommunicators,"Operate telephone, radio, or other communicati..."
779,472081,Drywall and Ceiling Tile Installers,Apply plasterboard or other wallboard to ceili...
1075,535031,Ship Engineers,Supervise and coordinate activities of crew en...


In [6]:
#C.4 Expert Casting Prompt
#From now on, you are an excellent {role} described as {roles_description}. Answer the following question while staying in strict accordance with the nature of the provided identity: {question}.
#url = "https://arxiv.org/pdf/2411.00492"

def create_agent(role, role_description):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="From now on, you are an excellent {} described as {}. you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible.".format(role, role_description),
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["-you are helpful and you think step by step."],
        add_history_to_messages=True,
        num_history_responses=3    
    )
    return expert_agent.run("use 1 short sentence to say who you are.").content, expert_agent.session_id

In [7]:
agents_in_action = {}
for role in range(len(sample_o)):
    agents_in_action[sample_o["name"].iloc[role]] = [create_agent(sample_o["name"].iloc[role], sample_o["description"].iloc[role])[1], sample_o["description"].iloc[role]]
agents_in_action

{'History Teachers, Postsecondary': ['a2d9f86a-c1c8-4fe7-90c5-accd2e4b782c',
  'Teach courses in human history and historiography. Includes both teachers primarily engaged in teaching and those who do a combination of teaching and research.'],
 'Teaching Assistants, Postsecondary': ['ab2e8b2d-3414-4364-95ba-f39852a740f4',
  'Assist faculty or other instructional staff in postsecondary institutions by performing instructional support activities, such as developing teaching materials, leading discussion groups, preparing and giving examinations, and grading examinations or papers. Graduate students who teach one or more full courses should be classified in the 25-1000 minor group.'],
 'Public Safety Telecommunicators': ['747eaa5f-c777-4f2f-9af2-8e0031325f12',
  'Operate telephone, radio, or other communication systems to receive and communicate requests for emergency assistance at 9-1-1 public safety answering points and emergency operations centers. Take information from the public and 

In [14]:
question = "Question: A ball is thrown vertically upward with an initial velocity of 20 m/s. Assuming g = 10 m/s², what is the maximum height reached by the ball? Options: A) 10 meters B) 20 meters C) 30 meters D) 40 meters Your answer: [Option]"


In [8]:
# ask agents questions
def ask_mc(question, session):
    expert_agent = Agent(
        model=Ollama(id=model_id),
        description="you are tasked with answering multiple choice questions. Think through each step meticulously but only show me the final answer. Rules: 1.Think deeply about the question's core concept. 2. Consider common misconceptions. 3. Mentally justify why each wrong option is incorrect 4. Double-check your logic 5. For calculation questions, solve using two different methods if possible. for output format, do this: [option (A, B, D, or D. do not repeat the answer text. alphbet only)]",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["""-you think step by step. -do not provide explanations. -you structure your answer in the following way: "questionX: the answer is X". """],
        add_history_to_messages=True,
        num_history_responses=3,
        session_id= session
    )
    return expert_agent.run(question).content

In [ ]:
# for agent in agents_in_action:
#     print(ask_mc(question, agents_in_action[agent]))

('The answer is D) Yellow.', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion1: the answer is C', '80b2d652-2772-4d44-8c28-1775e7f8cfc4')
('qustion1: the answer is C', 'f40c78f5-3a71-40af-b3c9-db9be14cdd06')
('C) Green', 'd652f109-ddf6-4c34-ba9d-6034885217e0')
('qustion1: the answer is C', 'bbd0b655-5515-4ce8-8f8c-1f83977178d8')


In [ ]:
# question = """Answer this multiple choice question. Think through each step meticulously but only show me the final answer.

# Question: In a population of 200 students, 60% are female. How many male students are in the population?

# Options:
# A) 80 students
# B) 120 students
# C) 140 students
# D) 160 students"""

In [ ]:
# question = """Question: Which of the following is NOT a primary color in traditional color theory?

# Options:
# A) Red
# B) Blue
# C) Green
# D) Yellow"""

In [ ]:
# with open("questions.json", "r") as file:
#     qb = json.load(file)

# for q in range(len(qb)):
#     print(qb[q]["question"],qb[q]["options"] )

# questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
# answers = [qb[q]["answer"] for q in range(len(qb))]
# questions

When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field? ['Angle the X-ray tube cephalad.', 'Position the patient in a prone position.', 'Align the X-ray beam perpendicular to the sternum.', 'Ask the patient to roll their shoulders forward.']
What is the recommended protocol when imaging a pediatric patient who cannot remain still? ['Use sedation to keep the patient immobile.', 'Use the shortest possible exposure time and immobilization devices.', 'Allow a parent to hold the child during the procedure without protective gear.', 'Perform the imaging procedure despite movement for efficiency.']
A fluoroscopy exam shows unusually high patient skin dose readings. What should you do? ['Stop the procedure immediately and report the incident.', 'Continue the procedure but decrease the fluoroscopy time.', 'Confirm equipment calibration and assess for overexposure risks.', 'Lower the table to reduce the distance between the X-ray source and

[['When positioning a patient for a chest X-ray, which adjustment ensures the clavicles are out of the lung field?',
  ['Angle the X-ray tube cephalad.',
   'Position the patient in a prone position.',
   'Align the X-ray beam perpendicular to the sternum.',
   'Ask the patient to roll their shoulders forward.']],
 ['What is the recommended protocol when imaging a pediatric patient who cannot remain still?',
  ['Use sedation to keep the patient immobile.',
   'Use the shortest possible exposure time and immobilization devices.',
   'Allow a parent to hold the child during the procedure without protective gear.',
   'Perform the imaging procedure despite movement for efficiency.']],
 ['A fluoroscopy exam shows unusually high patient skin dose readings. What should you do?',
  ['Stop the procedure immediately and report the incident.',
   'Continue the procedure but decrease the fluoroscopy time.',
   'Confirm equipment calibration and assess for overexposure risks.',
   'Lower the table

In [ ]:
prompt = """please design 10 multiple choice questions for this occupation. some thoughts before you repond: 
1. be highly domain specific. only test taker with over more than 30 years of experience are expected to answer correctly.
2. create nuance. design the answers in a way such that test takers cannot guess by intuition or inference .
3. do not limit yourself with only factual question. be creative.
strictly follow this example Json Output Structure:{"questions":[{"question":"[Insert question here]","answer_options":{"A":"[Option 1]","B":"[Option 2]","C":"[Option 3]","D":"[Option 4]"},"correct_answer":"A","explanation":"[This question tests specific skill/knowledge because of reason.]"}]}
"""

In [81]:
# ask agent to create mc
def mc_magician(role, role_des):
    prompt = """please design 10 multiple choice questions for {}. The job description is as follow: {}. Some thoughts before you repond: 
    1. be highly domain specific. only test taker with over more than 30 years of experience are expected to answer correctly.
    2. create nuance. design the answers in a way such that test takers cannot guess by intuition or inference .
    3. do not limit yourself with only factual question. be creative.
    4. do not response anything other than structed json output.""".format(role, role_des)

    mc_agent = Agent(
        model=Ollama(id=model_id),
        description=""""You are an expert question designer tasked with creating high-quality certification questions to identify true professionals.
        Your goal is to create 10 questions that:
        Test Practical Expertise: Focus on real-world scenarios, tools, and challenges specific to the domain.
        Assess Depth of Knowledge: Ensure questions range from fundamental concepts to advanced problem-solving, covering both theory and application.
        Differentiate Skill Levels: Include nuances that distinguish seasoned professionals from novices, such as best practices, edge-case scenarios, or optimized solutions.
        Simulate Professional Situations: Incorporate case studies or situational judgment problems that mimic day-to-day challenges professionals face.
        Reflect Industry Trends: Include questions that are relevant to current tools, techniques, and standards in the field.
        Each question should have a brief context or scenario.
        Structure questions only on advanced levels of difficulty
        output your questions in valid json format where new line is strictly prohitbited. only produce it in compact format. Strictly follow this example Json Output Structure:[{"question":"[Insert question here]","options":["[Option 1]","[Option 2]","[Option 3]","[Option 4]"],"answer":"[a/b/c/d]","reason":"[Explanation for the correct answer.]"},{"question":"[Insert question here]","options":["[Option 1]","[Option 2]","[Option 3]","[Option 4]"],"answer":"[a/b/c/d]","reason":"[Explanation for the correct answer.]"}]
        only output the required json. your output will be directly feed into other codes so only provide the clean data and nothing else.""",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information."],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return mc_agent.run(prompt).content, mc_agent.session_id

In [ ]:
# ask mc magician to create mc sets for each occupation
# manual effort for now
result_list = []
for i in trange(len(list(agents_in_action.items()))):
    result = mc_magician(list(agents_in_action.items())[i][0], list(agents_in_action.items())[0][1][1])
    result_list.append(result)
result_list

  0%|          | 0/5 [00:00<?, ?it/s]

In [74]:
agent_list = list(agents_in_action.items())
print(agent_list[1][0])
print(agent_list[1][1][1])
agents_in_action[agent][0]

Teaching Assistants, Postsecondary
Assist faculty or other instructional staff in postsecondary institutions by performing instructional support activities, such as developing teaching materials, leading discussion groups, preparing and giving examinations, and grading examinations or papers. Graduate students who teach one or more full courses should be classified in the 25-1000 minor group.


'16675b83-3d56-4f14-abb4-e4443178b582'

In [80]:
agents_in_action.keys()

dict_keys(['History Teachers, Postsecondary', 'Teaching Assistants, Postsecondary', 'Public Safety Telecommunicators', 'Drywall and Ceiling Tile Installers', 'Ship Engineers'])

In [82]:
result = mc_magician(role = agent_list[2][0], role_des=agent_list[2][1][1])
result

('[{"question":"A dispatcher receives a call from an intoxicated individual claiming to have been attacked by a large animal in the woods. The caller\'s speech is slurred, and they are unable to provide a specific location. Which action should the dispatcher take next?","options":["Ask for a detailed description of the attacker and their appearance.","Request that the caller remain on the line until help arrives, but do not send units yet.","Send a unit with basic life support training to assist the caller.","Tell the caller that they need to stay calm and provide more information before assistance can be dispatched."],"answer":"c","reason":"The dispatcher should prioritize sending a unit with basic life support training to assist the caller, as the situation may require medical attention due to the individual\'s intoxication. Requesting more information or delaying dispatch could exacerbate the situation."},{"question":"A 9-1-1 call is received from a residence where a family member h

In [83]:
qb = json.loads(result[0])
for q in range(len(qb)):
    print(qb[q]["question"],qb[q]["options"] )

questions = [[qb[q]["question"],qb[q]["options"]] for q in range(len(qb))]
answers = [qb[q]["answer"] for q in range(len(qb))]

A dispatcher receives a call from an intoxicated individual claiming to have been attacked by a large animal in the woods. The caller's speech is slurred, and they are unable to provide a specific location. Which action should the dispatcher take next? ['Ask for a detailed description of the attacker and their appearance.', 'Request that the caller remain on the line until help arrives, but do not send units yet.', 'Send a unit with basic life support training to assist the caller.', 'Tell the caller that they need to stay calm and provide more information before assistance can be dispatched.']
A 9-1-1 call is received from a residence where a family member has been shot. The caller provides a detailed description of the victim and the suspect, but they are unable to provide their location. The dispatcher must prioritize which piece of information to ask for next. ['Request that the caller remain on the line until help arrives.', 'Ask for a description of the room or building where the

In [84]:
print(questions)
print(answers)

[["A dispatcher receives a call from an intoxicated individual claiming to have been attacked by a large animal in the woods. The caller's speech is slurred, and they are unable to provide a specific location. Which action should the dispatcher take next?", ['Ask for a detailed description of the attacker and their appearance.', 'Request that the caller remain on the line until help arrives, but do not send units yet.', 'Send a unit with basic life support training to assist the caller.', 'Tell the caller that they need to stay calm and provide more information before assistance can be dispatched.']], ['A 9-1-1 call is received from a residence where a family member has been shot. The caller provides a detailed description of the victim and the suspect, but they are unable to provide their location. The dispatcher must prioritize which piece of information to ask for next.', ['Request that the caller remain on the line until help arrives.', 'Ask for a description of the room or buildin

In [33]:
def answer_cleaner(input):
    clean_agent = Agent(
        model=Ollama(id=model_id),
        description="you are a profession computer programmer with 20 years of experience. you are good at cleaning data and cextract answer to be a list of strings. you will receive input, and please extract answer from it. your output will be directly feed into other codes so only provide the clean data and nothing else.example: ['x','x','x','x','x','x'] where 'x' is either D, B, C, or A",
        storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent_storage.db"),
        instructions=["you only output what is reuqired and never provide explanation or additional information"],
        add_history_to_messages=True,
        num_history_responses=3,
        
    )
    return clean_agent.run(input).content

In [85]:
# ask the expert agents to answer the mc
for agent in agents_in_action:
    for q in range(len(questions)):
        print(ask_mc("""format is very important. remember to structure your result. no need to provide reaons. here is the question:"""+ str(questions[q]), agents_in_action[agent][0]))


question24: C
question25: B
question26: A
question27: C
question28: D
question29: D
question30: D
question31: D
question32: D
question33: C
question24: option C
question25: option B
question26: option A
question27: option C
question28: option D
question29: option D
question30: option D
question31: option D
question32: option D
question33: option C
Question24: The answer is C.
Question25: The answer is B
Question26: The answer is A
Question27: The answer is C
Question28: The answer is D
Question29: The answer is D
Question30: The answer is D
Question31: The answer is D
Question32: The answer is D
Question33: The answer is C
Question24: The answer is D.
Question25: The answer is B.
Question26: The answer is A.
Question27: The answer is C.
Question28: The answer is D.
Question29: The answer is D.
Question30: The answer is D.
Question31: The answer is D.
Question32: The answer is D.
Question33: The answer is C.
question24: option D)
question25: option B)
question26: option A)
question27: o

In [ ]:
c,b,a,c,d,d,d,d,d,c,
c,b,a,c,d,d,d,d,d,c,
c,b,a,c,d,d,d,d,d,c,
d,b,a,c,d,d,d,d,d,c,
d,b,a,c,d,d,d,d,d,c,

Financial Managers
Medical and Health Services Managers
Surgical Technologists
Hearing Aid Specialists
Agricultural Inspectors


In [81]:
print('b', 'a', 'c', 'c', 'd', 'c', 'd', 'd', 'c', 'c')

b a c c d c d d c c


In [1]:
messydata = """('qustion:1, answer:A', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:2, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:3, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:4, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:5, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:6, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:7, answer:C', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:8, answer:D', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:9, answer:A', '39f281cb-5653-4c53-aa71-93693110ec63')
('qustion:10, answer:B', '39f281cb-5653-4c53-aa71-93693110ec63')"""

In [ ]:
eval(answer_cleaner(messydata)[0])
#use json.loads

['A', 'B', 'C', 'C', 'C', 'C', 'B', 'C', 'D', 'A', 'B']